# Working with distributed

[Distributed](https://distributed.readthedocs.org) is a cool library for doing distributed execution. You should check it out, if you haven't already.

Assuming you already have an IPython cluster running:

In [1]:
import ipyparallel as ipp
rc = ipp.Client()
rc.ids

[0, 1, 2, 3, 4, 5, 6, 7]

You can turn your IPython cluster into a distributed cluster by calling `Client.become_distributed()`:

In [2]:
executor = rc.become_distributed(ncores=1)
executor

<Executor: scheduler=172.16.3.46:63868 workers=8 threads=8>

This will:

1. start a Scheduler on the Hub
2. start a Worker on each engine
3. return an Executor, the distributed client API

By default, distributed Workers will use threads to run on all cores of a machine. 
In this case, since I already have one *engine* per core,
I tell distributed to run one core per Worker with `ncores=1`.

We can now use our IPython cluster with distributed:

In [3]:
from distributed import progress

def square(x):
    return x ** 2

def neg(x):
        return -x

A = executor.map(square, range(1000))
B = executor.map(neg, A)
total = executor.submit(sum, B)
progress(total)

In [4]:
total.result()

-332833500